In [43]:
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, train_test_split
from collections import Counter
import pandas as pd
import os

annotation = "/opt/ml/dataset/train.json"

# json 파일 열어서 데이터를 불러온다.
with open(annotation) as f:
    data = json.load(f)

# 이미지 id와 카테고리 id(클래스)를 리스트에 저장한다.
var = [(ann["image_id"], ann["category_id"]) for ann in data["annotations"]]

X = np.ones((len(data["annotations"]), 1))  # 모든 데이터 포인트에 대해 1로 이루어진 X 배열을 만든다.
y = np.array([v[1] for v in var])  # 각 데이터 포인트의 카테고리 id(클래스)를 y 배열에 저장한다.
groups = np.array([v[0] for v in var])  # 각 데이터 포인트의 이미지 id를 groups 배열에 저장한다.

# StratifiedGroupKFold로 데이터를 5개로 나눠 교차 검증을 진행한다.
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=18)

# 각 split에서 train, validation 데이터의 이미지 id와 카테고리 id를 출력한다.
# for train_idx, val_idx in cv.split(X, y, groups):
#     print("TRAIN:", groups[train_idx])
#     print("      ", y[train_idx])
#     print(" TEST:", groups[val_idx])
#     print("      ", y[val_idx])

코드는 다음과 같이 작동합니다:

1. train.json 파일을 엽니다.
2. annotations에서 각 이미지와 해당 이미지에 대한 주석(annotation)을 가져와서 이미지 ID와 카테고리 ID를 저장합니다.
3. 모든 데이터 포인트에 대해 1로 이루어진 배열 X를 생성합니다.
4. 각 데이터 포인트의 카테고리 ID를 y에 저장합니다.
5. 각 데이터 포인트의 이미지 ID를 groups에 저장합니다.
6. StratifiedGroupKFold를 사용하여 데이터를 5개로 나눠 교차 검증을 진행합니다.
7. 각 split에서 train, validation 데이터의 이미지 ID와 카테고리 ID를 출력합니다.

즉, 이 코드는 이미지 분류 모델을 학습시키기 위한 데이터를 준비하는 단계 중 하나인 교차 검증을 수행합니다. 

교차 검증을 통해 모델의 일반화 성능을 평가할 수 있습니다. StratifiedGroupKFold를 사용하여 데이터를 그룹화하고, 각 fold에서 각 그룹의 비율을 고려하여 데이터를 분할합니다. 

이를 통해 모델이 특정 그룹에 대해 overfitting되는 것을 방지할 수 있습니다. 출력된 결과를 통해 각 fold에서 train 데이터와 validation 데이터의 이미지 ID와 카테고리 ID를 확인할 수 있습니다.

X 배열은 y와 groups 배열과 같은 shape를 갖는 배열로, 모든 데이터 포인트에 대해 1로 이루어진 배열입니다.

이 배열은 StratifiedGroupKFold 함수를 사용할 때, 첫 번째 인자로서 데이터를 전달해야 하기 때문에 필요합니다. 

X 배열은 이 함수에서는 사용되지 않고, 학습 데이터와 검증 데이터를 나눌 때 y와 groups 정보만 사용됩니다. 따라서 X 배열은 각 데이터 포인트에 대해 동일한 가중치를 부여하고자 하는 의미 없는 배열이라고 할 수 있습니다.


In [44]:
# get_distribution 함수: 주어진 y 값의 분포를 계산하여 반환하는 함수
def get_distribution(y):
    y_distr = Counter(y)  # y 값의 개수를 세어 Counter 객체로 반환
    y_vals_sum = sum(y_distr.values())  # y 값의 총 개수 계산

    # 각 y 값이 전체 중 어느 정도의 비율을 차지하는지 계산하여 문자열 리스트로 반환
    return [f"{y_distr[i]/y_vals_sum:.2%}" for i in range(np.max(y) + 1)]


# 초기화
distrs = [get_distribution(y)]  # y의 분포 계산하여 리스트에 저장
index = ["training set"]  # 인덱스로 사용할 리스트 초기화

# 교차 검증(fold) 수행 및 분포 계산
for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]  # 교차 검증에 사용할 train, val 데이터 분리
    train_gr, val_gr = groups[train_idx], groups[val_idx]  # 각 데이터의 그룹 정보 분리

    # 각 fold에서 train, validation 데이터의 y 분포 계산하여 리스트에 저장
    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))

    # 인덱스 리스트에 train, val fold 정보 추가
    index.append(f"train - fold{0}")
    index.append(f"val - fold{0}")

categories = [d["name"] for d in data["categories"]]  # 데이터셋의 카테고리 이름 리스트

# 분포 결과를 데이터프레임으로 변환하여 출력
pd.DataFrame(distrs, index=index, columns=[categories[i] for i in range(np.max(y) + 1)])

,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
train - fold0,17.11%,27.15%,3.87%,3.92%,4.16%,13.16%,5.35%,22.36%,0.76%,2.16%
val - fold0,17.25%,28.57%,3.90%,4.50%,4.57%,11.05%,5.85%,22.41%,0.39%,1.51%
train - fold0,17.03%,27.94%,3.66%,4.04%,4.44%,12.81%,5.41%,21.83%,0.76%,2.09%
val - fold0,17.59%,25.31%,4.82%,4.07%,3.40%,12.31%,5.66%,24.71%,0.39%,1.74%
train - fold0,17.12%,27.41%,3.91%,4.15%,4.23%,12.83%,5.55%,22.28%,0.63%,1.88%
val - fold0,17.18%,27.59%,3.74%,3.64%,4.28%,12.28%,5.10%,22.72%,0.91%,2.55%
train - fold0,17.44%,26.95%,4.12%,4.07%,4.30%,12.50%,5.46%,22.55%,0.55%,2.05%
val - fold0,15.96%,29.33%,2.95%,3.93%,4.02%,13.54%,5.46%,21.69%,1.21%,1.90%
train - fold0,16.98%,27.75%,3.83%,4.04%,4.08%,12.29%,5.51%,22.84%,0.73%,1.93%


이 코드는 교차 검증(cross-validation)을 통해 이미지 분류 모델을 학습하기 전, 데이터셋의 레이블 분포를 확인하는 함수입니다.

get_distribution 함수는 레이블(y)의 분포를 계산하고, 각 레이블의 비율을 문자열 형태로 반환합니다.

distrs 리스트에는 먼저 전체 데이터셋의 레이블 분포를 저장합니다. index 리스트에는 각 분할(fold)의 이름을 저장합니다.

cv.split 함수를 이용하여 데이터를 분할하고, 각 분할에서 train과 validation 데이터의 레이블 분포를 계산하여 distrs 리스트에 추가합니다. index 리스트에는 각 fold의 이름을 추가합니다.

마지막으로 pd.DataFrame 함수를 이용하여 distrs 리스트를 데이터프레임으로 변환합니다. 열(column) 이름은 레이블의 이름으로, 행(row) 이름은 index 리스트의 요소로 설정됩니다.

In [45]:
# 데이터셋 분할을 위한 KFold CV를 사용하여 학습 및 검증 데이터를 나눔
output_path = "/opt/ml/dataset/stest"
if not os.path.exists(output_path):
    os.makedirs(output_path)

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    # 학습 및 검증 데이터에 맞는 annotation 분리
    train_annotations = [ann for i, ann in enumerate(data["annotations"]) if i in train_idx]
    val_annotations = [ann for i, ann in enumerate(data["annotations"]) if i in val_idx]

    # 선택된 annotation을 가진 이미지 id를 찾음
    train_image_ids = set([ann["image_id"] for ann in train_annotations])
    val_image_ids = set([ann["image_id"] for ann in val_annotations])

    # 학습 및 검증 데이터에 맞는 이미지 분리
    train_images = [img for img in data["images"] if img["id"] in train_image_ids]
    val_images = [img for img in data["images"] if img["id"] in val_image_ids]

    # 학습 및 검증 데이터에 맞는 dictionary 생성
    train_data = {"images": train_images, "annotations": train_annotations, "categories": data["categories"]}
    val_data = {"images": val_images, "annotations": val_annotations, "categories": data["categories"]}

    # 학습 및 검증 데이터를 각각의 json 파일로 저장
    train_json_path = os.path.join(output_path, f"train_{fold_ind}.json")
    with open(train_json_path, "w") as f:
        json.dump(train_data, f, indent=4)

    val_json_path = os.path.join(output_path, f"val_{fold_ind}.json")
    with open(val_json_path, "w") as f:
        json.dump(val_data, f, indent=4)

# 학습 및 검증 데이터를 로드하여 데이터 유효성 검사
for i in range(5):
    train_json_path = os.path.join(output_path, f"train_{fold_ind}.json")
    val_json_path = os.path.join(output_path, f"val_{fold_ind}.json")

    with open(train_json_path, "r") as f:
        train_data = json.load(f)
    with open(val_json_path, "r") as f:
        val_data = json.load(f)

    # 학습 데이터와 검증 데이터 총 이미지 개수가 4883개인지 검사
    assert len(train_data["images"]) + len(val_data["images"]) == 4883, "데이터 유효성 검사 실패"

print("데이터가 모두 저장되었습니다")

데이터가 모두 저장되었습니다


위 코드는 KFold CV를 사용하여 데이터셋을 분할하고 각각을 학습과 검증용으로 구분한 후, json 파일로 저장하는 과정을 나타낸다. 마지막으로 저장된 데이터를 다시 로드하여 데이터 유효성을 검사하는 코드까지 포함되어 있다.

In [46]:
# 훈련셋과 테스트셋으로 분리
train_idx, test_idx = train_test_split(np.arange(len(data["images"])), test_size=0.2, shuffle=True, random_state=18)
print(len(train_idx), len(test_idx))

# 클래스 분포 확인을 위해 리스트 초기화
distrs = [get_distribution(y)]
index = ["training set"]

# 훈련셋, 검증셋으로 분리하여 각각 클래스 분포 확인
train_y, val_y = y[train_idx], y[val_idx]
train_gr, val_gr = groups[train_idx], groups[val_idx]

distrs.append(get_distribution(train_y))
distrs.append(get_distribution(val_y))

index.append(f"train - fold{0}")
index.append(f"val - fold{0}")

# 카테고리명 추출
categories = [d["name"] for d in data["categories"]]

# 데이터프레임 생성하여 출력
pd.DataFrame(distrs, index=index, columns=[categories[i] for i in range(np.max(y) + 1)])

3906 977


,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
train - fold0,17.92%,26.63%,3.48%,4.61%,4.07%,12.49%,4.33%,23.66%,1.23%,1.59%
val - fold0,17.81%,26.09%,4.06%,4.08%,4.95%,14.59%,5.20%,20.30%,0.49%,2.43%


이 코드는 데이터를 학습셋과 검증셋으로 나누고, 각 셋의 클래스 분포를 확인하기 위해 Pandas DataFrame을 사용하여 표로 출력하는 코드입니다.

우선 train_test_split 함수를 사용하여 전체 데이터를 학습셋과 검증셋으로 나눕니다. 그리고 get_distribution 함수를 사용하여 클래스 분포를 계산하고, 이를 distrs 리스트에 저장합니다.

그 다음, index 리스트에는 각 셋의 이름을 저장하고, categories 리스트에는 데이터셋에서 사용된 클래스의 이름을 저장합니다. 마지막으로, pd.DataFrame 함수를 사용하여 distrs 리스트를 Pandas DataFrame으로 변환하고, 셋의 이름과 클래스 이름을 이용하여 표로 출력합니다.

In [47]:
output_path = "/opt/ml/dataset"  # json 파일을 저장할 경로
train_data = {"images": [], "annotations": [], "categories": data["categories"]}  # train set을 위한 dictionary
test_data = {"images": [], "annotations": [], "categories": data["categories"]}  # test set을 위한 dictionary

# train_idx와 test_idx를 기준으로 이미지 데이터를 train set과 test set으로 분리합니다.
for i in train_idx:
    image = data["images"][i]
    train_data["images"].append(image)
    for ann in data["annotations"]:
        if ann["image_id"] == image["id"]:
            train_data["annotations"].append(ann)

for i in test_idx:
    image = data["images"][i]
    test_data["images"].append(image)
    for ann in data["annotations"]:
        if ann["image_id"] == image["id"]:
            test_data["annotations"].append(ann)

# train set과 test set을 각각 json 파일로 저장합니다.
train_json_path = os.path.join(output_path, "train_.json")
with open(train_json_path, "w") as f:
    json.dump(train_data, f, indent=4)

test_json_path = os.path.join(output_path, "val_.json")
with open(test_json_path, "w") as f:
    json.dump(test_data, f, indent=4)

output_path 변수는 json 파일이 저장될 경로를 지정합니다. train_data와 test_data 변수는 각각 train set과 test set을 저장하기 위한 dictionary 입니다. 

train_idx와 test_idx는 데이터셋의 이미지를 분리하기 위한 인덱스이며, 이를 이용하여 이미지 데이터를 train set과 test set으로 분리합니다. 그 후, 분리된 데이터셋을 json 파일로 저장합니다.